# GETTIN
## Planejamento de Equipamentos Escolares
## Método: k-Median Facility Location
### Versão: 2.0
#### Fillipe O Feitosa <fillipefeitosa@ua.pt>

---


\begin{split}& \mbox{minimize} \quad    & \sum_{i=1}^n \sum_{j=1}^m d_{ij} y_{ij} &     \\
& \mbox{subject to:} \quad & \sum_{j=1}^m y_{ij} =1 &  \mbox{ for } i=1,\cdots,n\\
&   & \sum_{j=1}^m x_{j} = k                        &   \\
&   & y_{ij} \leq x_j                               & \mbox{ for }  i=1,\cdots,n; j=1,\cdots,m  \\
&   & y_{ij} \in \{ 0,1 \}                          & \mbox{ for }  i=1,\cdots,n; j=1,\cdots,m \\
&   & x_j \in \{ 0,1 \}                             & \mbox{ for }  j=1,\cdots,m\end{split}

\begin{split}&   & \sum_{i=1}^{n} y_{ij} \leq y_j,                               & \mbox{ for }  j=1,\cdots,m.  \\\end{split}

#### Modeling and Creating Decision Variables

In [ ]:
# Import Libraries
from gurobi import *
import math
import matplotlib.pyplot as plt

# Used to Create Visualization
from geojson import Feature, Point, FeatureCollection
import geopandas as gpd
import geojson

def distance(a,b):
    dx = a[0] - b[0]
    dy = a[1] - b[1]
    return math.sqrt(dx*dx + dy*dy)

# Data
# Problem data

k = int(input("Provide a Int K: "))

vagos = gpd.read_file('./data_gettin/vagos.geojson')
centroids = vagos.centroid

iteratorHandler = centroids.size
centroidVector = []
for centroid in centroids:
    obj = [centroid.xy[0][0], centroid.xy[1][0]]
    centroidVector.append(obj)

subSections = centroidVector
schools = centroidVector
# @charge: custo de construcao por escola (fixo em 500 mil euros)
# charge = 5000000

numSchools = len(schools)
numSubSections = len (subSections)

# Creting Guroby Model
m = Model()

# Decision Variables
x = {}
y = {}
d = {} # Distance Matrix
# @alpha: 0.29 de custo por Km  por (365 dias * 5 anos) 
# alpha = 529.25

# creating binary variable for every school
for j in range(numSchools):
    x[j] = m.addVar(obj=0, vtype=GRB.BINARY, name="%d" % j)

# creating continuous variable for subsections to check suply fractions
for i in range(numSubSections):
    for j in range(numSchools):
        # Distances between Subsections and Schools
        d[(i,j)] = distance(subSections[i], schools[j])
        # Fractions of Subsection Suply
        y[(i,j)] = m.addVar(obj=d[i,j], vtype=GRB.BINARY, name="Fração da Sub[%d], escola[%d]" % (i,j))
        

m.update()

In [ ]:
subSections

In [ ]:
plt.scatter(*zip(*subSections))
plt.show()

In [ ]:
plt.scatter(*zip(*schools))
plt.show()

## Adding Constraints

In [ ]:
# Constraint for Every Student on School
# for i in range(numSubSections):
#     m.addConstr(quicksum(y[(i,j)] for j in range(numSchools)) == 1)

# Fraction of suply must be lower or equal than the BINARY x[j] available 
# for i in range(numSubSections):
#     for j in range(numSchools):
#         m.addConstr(y[(i,j)] <= x[j])

for i in range(numSubSections):
    coef = [1 for j in range(numSchools)]
    var = [y[i,j] for j in range(numSchools)]
    m.addConstr(LinExpr(coef,var), "=", 1, name="Assign[%s]"%i)

for j in range(numSchools):
    for i in range(numSubSections):
        m.addConstr(y[i,j], "<", x[j], name="Strong[%s,%s]"%(i,j))

coef = [1 for j in range(numSchools)]
var = [x[j] for j in range(numSchools)]
m.addConstr(LinExpr(coef,var), "=", rhs=k, name="k_median")

#### Objetive


In [ ]:
# Setting objective

m.setObjective(quicksum(d[(i,j)]*y[(i,j)] for i in range(numSubSections) for j in range(numSchools)), GRB.MINIMIZE)


m.optimize()

In [ ]:
m

In [ ]:
print('Obj: %g' % m.objVal)

In [ ]:
# Added @listOfResults to generate visualization

for v in m.getVars():
    if(v.x != 0):
        print('%s   %g' % (v.varName, v.x))

In [ ]:
# How to retrieve variables from the Model by Name
# int(m.getVarByName("19").varName)

# Used to get the Index of Schools from the M Model Optimized
listOfResults = []
for e in range(numSchools):
    tempObj = m.getVarByName(str(e))
    # If This School is on the Results Optimized
    if(tempObj.x != 0):
        listOfResults.append(int(tempObj.varName))
        
# Select, from the List Of Results, A set of Centroid Points 
listOfPoints = []
for schoolNumber in listOfResults:
    # Attention to the Feature(geometry) from geopandas
    listOfPoints.append(Feature(geometry=centroids[schoolNumber]))

# Creating a FeatureCollection with the Features (Points) manipulated above
resultCentroids = FeatureCollection(listOfPoints)

In [ ]:
# Write Feature Collection to GeoJson File
dump = geojson.dumps(resultCentroids, sort_keys=True)
with open('./data_gettin/k-median-%d.geojson' % k, 'w') as outfile:
      geojson.dump(resultCentroids, outfile)

In [ ]:
m = None

In [ ]:
disposeDefaultEnv()